In [68]:
from dgl.nn import GATConv

class GAT(nn.Module):
    def __init__(self, g, in_dim, hidden_dim, out_dim, num_heads):
        super(GAT, self).__init__()
        self.g = g
        self.layer1 = GATConv(in_dim, hidden_dim, num_heads)
        self.layer2 = GATConv(hidden_dim * num_heads, out_dim, 1)

    def forward(self, h):
        h = self.layer1(self.g, h)
        h = F.elu(h)
        h = h.flatten(1)
        h = self.layer2(self.g, h)
        return h

In [69]:
from dgl import DGLGraph
from dgl.data import citation_graph as citegrh
import networkx as nx

def load_cora_data():
    data = citegrh.load_cora()
    features = torch.FloatTensor(data.features)
    labels = torch.LongTensor(data.labels)
    mask = torch.BoolTensor(data.train_mask)
    g = DGLGraph(data.graph)
    return g, features, labels, mask

In [79]:
import time
import numpy as np

g, features, labels, mask = load_cora_data()

# create the model, 2 heads, each head has hidden size 8
net = GAT(g,
          in_dim=features.size()[1],
          hidden_dim=8,
          out_dim=7,
          num_heads=2)

# create optimizer
optimizer = torch.optim.Adam(net.parameters(), lr=1e-3)

# main loop
dur = []
for epoch in range(30):
    if epoch >= 3:
        t0 = time.time()

    logits = net(features)
    print(mask.shape)
    print(logits.shape)
#     loss = F.cross_entropy(logits[mask], labels[mask]) 
#     print("mask.shape: ", mask.shape)
    print("labels.shape: ", labels.shape)
    logp = F.log_softmax(logits, 1)
    print(logp.shape)
    loss = F.nll_loss(logp[mask.unsqueeze(-1).unsqueeze(-1).repeat(1,1,7)], labels[mask])
#     loss = F.cross_entropy(logp[mask], labels[mask])

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if epoch >= 3:
        dur.append(time.time() - t0)

    print("Epoch {:05d} | Loss {:.4f} | Time(s) {:.4f}".format(
        epoch, loss.item(), np.mean(dur)))

  NumNodes: 2708
  NumEdges: 10556
  NumFeats: 1433
  NumClasses: 7
  NumTrainingSamples: 140
  NumValidationSamples: 500
  NumTestSamples: 1000
Done loading data from cached files.
torch.Size([2708])
torch.Size([2708, 1, 7])
labels.shape:  torch.Size([2708])
torch.Size([2708, 1, 7])


RuntimeError: size mismatch (got input: [980], target: [140])